## Project 5 : Artificial Neural Network
* CS 5300 : Artificial Intelligent
* Kyu Cho
* 5/8/16

## Data Description
### Intro
The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

### Variables
Note: This data already has been preprocessed.


**Sex** : Sex (1 = Female; 0 = Male)  
**FamilySize** : Family (1 = Aboard with Family; 0 = Aboard without Family)  
**Child** : Child (age < 16) (1 = Child; 0 = Not Child)  
**Pclass.2** : Passenger class (1 = 2nd Class; 0 = 1st or 3rd Class)  
**Pclass.3** : Passenger class (1 = 3rd Class; 0 = 1st or 2nd Class)  
Note : It does not have value for 1st class because if Pclass.2 = 0 and Pclass.3 = 0 meaning it's Pclass.1.  
**Survive** : Survival (0 = No; 1 = Yes)  

## Implementation
Here is the outline of this program.
 * Convert an Datafrane into a NumPy array.
 * Implement the link function for ANN.
 * Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
 * Implement gradient ascent.
 * Given a set of coefficients, predict survive.
 * Compute classification accuracy for the ANN model.

## Load data

In [9]:
import pandas as pd
import numpy as np
import math
import os
os.chdir('C:\Users\Kyu\Documents\python\data')

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.head())

features = ['Sex',
            'FamilySize',
            'Child',
            'Pclass.2',
            'Pclass.3']
target = 'Survived'
train[train.Survived == 0] = -1
test[test.Survived == 0] = -1
print ('# of Alives =', len(train[train['Survived'] == 1]))
print ('# of Deads =', len(train[train['Survived'] == -1]))

   Sex  FamilySize  Child  Pclass.2  Pclass.3  Survived
0    0           0      0         0         1         0
1    1           0      0         0         0         1
2    1           0      0         0         0         1
3    0           0      0         0         1         0
4    0           0      0         0         0         0
('# of Alives =', 278)
('# of Deads =', 434)


## Convert dataframe into NumPy array

Two arrays are returned: one representing features and another representing class labels. 
Note that the feature matrix includes an additional column 'intercept' to take account of the intercept term.

In [10]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features # Adding intercept variables in to the features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.as_matrix()
    label_sarray = data_sframe[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

In [16]:
# Convert the data into NumPy arrays
feature_matrix, survived = get_numpy_data(train, features, 'Survived') 

## Compute conditional probability with link function
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the value of **each feature**  $\mathbf{x}_i$. Complete the following function that implements the link function:

In [17]:
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    dot_product = np.dot(feature_matrix, coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    predictions = 1/(1+np.exp(-dot_product))

    # return predictions
    return predictions

**Aside**. How the link function works with matrix algebra

Since the each features are stored as columns in **feature_matrix**, each $i$-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\
h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N)
\end{array}
\right]
$$

By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^T h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$.
$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
h(\mathbf{x}_1)^T\mathbf{w} \\
h(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
h(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

## Compute derivative of log likelihood with respect to a single coefficient

$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

We will now write a function that computes the derivative of log likelihood with respect to a single coefficient $w_j$. The function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 


In [18]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)
    
    # Return the derivative
    return derivative

## Compute log likelihood

The log likelihood simplifies the derivation of the gradient and is more numerically stable.  Due to its numerical stability, we will use the log likelihood instead of the likelihood to assess the algorithm.

The log likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$

We provide a function to compute the log likelihood for the entire dataset. 

In [19]:
def compute_log_likelihood(feature_matrix, survived, coefficients):
    indicator = (survived==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]
    
    lp = np.sum((indicator-1)*scores - logexp)
    return lp

## Implement gradiant decent algorithm

In [20]:
from math import sqrt

def ann(feature_matrix, survived, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (survived==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        new_coefficients = np.ndarray(shape=coefficients.shape)
        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            derivative = feature_derivative(errors, feature_matrix[:, j])
            
            # add the step size times the derivative to the current coefficient
            new_coefficients[j] = coefficients[j] + step_size * derivative
        
        coefficients = new_coefficients
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, survived, coefficients)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                 (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

## Build the model

In [23]:
coefficients = ann(feature_matrix, survived, initial_coefficients=np.zeros(6),
                                   step_size=1e-7, max_iter=801)

iteration   0: log likelihood of observed labels = -493.48475371
iteration   1: log likelihood of observed labels = -493.44871947
iteration   2: log likelihood of observed labels = -493.41268984
iteration   3: log likelihood of observed labels = -493.37666482
iteration   4: log likelihood of observed labels = -493.34064441
iteration   5: log likelihood of observed labels = -493.30462860
iteration   6: log likelihood of observed labels = -493.26861740
iteration   7: log likelihood of observed labels = -493.23261081
iteration   8: log likelihood of observed labels = -493.19660882
iteration   9: log likelihood of observed labels = -493.16061143
iteration  10: log likelihood of observed labels = -493.12461865
iteration  11: log likelihood of observed labels = -493.08863047
iteration  12: log likelihood of observed labels = -493.05264690
iteration  13: log likelihood of observed labels = -493.01666792
iteration  14: log likelihood of observed labels = -492.98069355
iteration  15: log likeli

## Predict the output

Class predictions for a data point $\mathbf{x}$ can be computed from the coefficients $\mathbf{w}$ using the following formula:
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

Now, we will write some code to compute class predictions. We will do this in two steps:
* **Step 1**: First compute the **scores** using **feature_matrix** and **coefficients** using a dot product.
* **Step 2**: Using the formula above, compute the class predictions from the scores.

Step 1 can be implemented as follows:

In [30]:
# Step 1
scores = np.dot(feature_matrix, coefficients)
scores[1:10]

array([ 0.01830103,  0.01830103, -0.10945666, -0.10945666, -0.10945666,
        0.05911417,  0.05692958,  0.07802963,  0.01830103])

In [31]:
# Step 2
npa_possent = (scores > 0)
npa_possent = npa_possent[npa_possent == True]
npa_possent[1:10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

## Accuracy

The classification accuracy can be computed as follows:

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

In [36]:
predictions = np.empty_like(scores)
for i in range(len(scores)):
    predictions[i] = +1 if scores[i]> 0 else -1

measures = predictions == train['Survived'] 
num_mistakes = len(measures[measures == False])
accuracy = len(measures[measures == True]) / float(len(measures)) 
print("-----------------------------------------------------")
print('# Correctly classified   =', len(train) - num_mistakes)
print('# Incorrectly classified =', num_mistakes)
print('# Total                  =', len(train))
print("-----------------------------------------------------")
print('Accuracy = %.2f' % accuracy)

-----------------------------------------------------
('# Correctly classified   =', 679)
('# Incorrectly classified =', 33)
('# Total                  =', 712)
-----------------------------------------------------
Accuracy = 0.95
